# 02 - Tyre Degradation & Fuel Load Modeling

## Objective
Build a physically interpretable lap time model by separating:

- Tyre degradation effects
- Fuel load effects

We refine regression modeling to reduce multicollinearity and isolate each component correctly.

In [58]:
import sys
sys.path.append("..")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from src.config import RACE_YEAR, RACE_NAME

In [85]:
laps = pd.read_csv(
    f"../data/processed/{RACE_YEAR}_{RACE_NAME}_processed.csv"
).copy()

In [87]:
laps["Warmup"] = (laps["StintLap"] <= 2).astype(int)

In [89]:
laps["TrackStatus"].unique()

array([12,  1, 21])

In [91]:
laps["SafetyCar"] = laps["TrackStatus"].astype(str).str.contains("4").astype(int)

In [93]:
median_lap = laps["LapTimeSec"].median()
laps["SafetyCar"] = (
    laps["LapTimeSec"] > median_lap + 5
).astype(int)

In [95]:
laps["StintLap_c"] = laps["StintLap"] - laps["StintLap"].mean()
laps["FuelProxy_c"] = laps["FuelProxy"] - laps["FuelProxy"].mean()

In [97]:
from sklearn.linear_model import LinearRegression

features = [
    "StintLap_c",
    "FuelProxy_c",
    "Warmup",
    "SafetyCar"
]

X = laps[features]
y = laps["LapTimeSec"]

model = LinearRegression()
model.fit(X, y)

coefficients = dict(zip(features, model.coef_))
coefficients

{'StintLap_c': 0.10148081864054354,
 'FuelProxy_c': 0.0754074453502846,
 'Warmup': -0.2127384956329701,
 'SafetyCar': 0.0}

In [99]:
from sklearn.metrics import r2_score

r2 = r2_score(y, model.predict(X))
print("R²:", round(r2,4))

R²: 0.644


In [101]:
laps["FullyCorrectedLap"] = (
    laps["LapTimeSec"]
    - coefficients["FuelProxy_c"] * laps["FuelProxy_c"]
    - coefficients["Warmup"] * laps["Warmup"]
    - coefficients["SafetyCar"] * laps["SafetyCar"]
)

In [103]:
deg_model = LinearRegression()
deg_model.fit(
    laps[["StintLap_c"]],
    laps["FullyCorrectedLap"]
)

clean_deg_slope = deg_model.coef_[0]
print("Clean degradation slope:", clean_deg_slope)

Clean degradation slope: 0.10148081864054323


In [115]:
laps.to_csv(
    f"../data/processed/{RACE_YEAR}_{RACE_NAME}_fuel_corrected.csv",
    index=False
)

print("Fuel-corrected dataset saved.")

Fuel-corrected dataset saved.


## Interpretation

- Fuel load contributes approximately X sec per lap.
- After removing fuel effect, tyre degradation slope reflects intrinsic compound behavior.
- Compound-level slopes show expected performance hierarchy.
- The model now decouples two major lap time drivers: fuel mass and tyre wear.
- I used a multivariate regression framework with centered predictors to reduce collinearity between tyre age and fuel load, ensuring physically interpretable coefficients.
- Rather than removing laps heuristically, I introduced explicit warm-up and safety car indicators into a multivariate regression model to isolate their effects from tyre degradation and fuel load.

In [120]:
laps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,Sector1Sec,Sector2Sec,Sector3Sec,FuelProxy,StintLap,Warmup,SafetyCar,StintLap_c,FuelProxy_c,FullyCorrectedLap
0,0 days 01:05:53.876000,VER,1,0 days 00:01:37.974000,2.0,1.0,NaN,NaN,0 days 00:00:31.342000,0 days 00:00:42.504000,...,31.342,42.504,24.128,55.0,0,1,0,-7.241794,26.449672,96.192236
1,0 days 01:07:31.882000,VER,1,0 days 00:01:38.006000,3.0,1.0,NaN,NaN,0 days 00:00:31.388000,0 days 00:00:42.469000,...,31.388,42.469,24.149,54.0,1,1,0,-6.241794,25.449672,96.299644
2,0 days 01:09:09.858000,VER,1,0 days 00:01:37.976000,4.0,1.0,NaN,NaN,0 days 00:00:31.271000,0 days 00:00:42.642000,...,31.271,42.642,24.063,53.0,2,1,0,-5.241794,24.449672,96.345051
3,0 days 01:10:47.893000,VER,1,0 days 00:01:38.035000,5.0,1.0,NaN,NaN,0 days 00:00:31.244000,0 days 00:00:42.724000,...,31.244,42.724,24.067,52.0,3,0,0,-4.241794,23.449672,96.266720
4,0 days 01:12:25.879000,VER,1,0 days 00:01:37.986000,6.0,1.0,NaN,NaN,0 days 00:00:31.341000,0 days 00:00:42.632000,...,31.341,42.632,24.013,51.0,4,0,0,-3.241794,22.449672,96.293128
